In [1]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, trim_messages
from langchain_community.llms import Ollama

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter
import json

from langchain_core.runnables import RunnablePassthrough

model = Ollama(model="llama2")

# model.invoke([HumanMessage(content="Hi! I'm Bob")])

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


with open('ChatTest.chat') as f:
    content = json.load(f)

last_processed_message = content["messages"][-1]

messages = []
for message in content['messages']:
    if 'automated' in message:
        messages.append(AIMessage(content=message['body']))
    else:
        messages.append(HumanMessage(content=message['body']))

trimmer = trim_messages(
    max_tokens=120,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=True,
    # start_on="human",
)

# trimmer.invoke(messages)

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="I actually need the max value")],            
    }
)
response

"\nSure, I'd be happy to help! To calculate the maximum value in a column in Pandas, you can use the `max()` function. Here is an example of how you can use it:\n```\nimport pandas as pd\n\n# create a sample dataset\ndf = pd.DataFrame({'weight': [6, 4, 8, 10, 6]})\n\n# calculate the maximum value in the 'weight' column\nmax_value = df['weight'].max()\n\nprint(max_value) # Output: 10\n```\nIn this example, the `max()` function is applied to the `weight` column of the `df` dataset. This will return the maximum value in that column, which in this case is 10.\n\nI hope this helps! Let me know if you have any other questions."

: 

In [16]:
with open('ChatTest.chat') as f:
    content = json.load(f)

In [29]:
for user in content['users']:
    print(user)

1227f3c151714816a1282be285864518


In [38]:
content['messages'][0].get('sender')

'1227f3c151714816a1282be285864518'

In [ ]:
if len(content['users']) == 1:
    

In [2]:
# model.invoke(
#     [
#         HumanMessage(content="Hi! I'm Bob"),
#         AIMessage(content="Hello Bob! How can I assist you today?"),
#         HumanMessage(content="What's my name?"),
#     ]
# )

In [3]:
# from langchain_core.chat_history import (
#     BaseChatMessageHistory,
#     InMemoryChatMessageHistory,
# )
# from langchain_core.runnables.history import RunnableWithMessageHistory

# store = {}

# def get_session_history(session_id: str) -> BaseChatMessageHistory:
#     if session_id not in store:
#         store[session_id] = InMemoryChatMessageHistory()
#     return store[session_id]


# with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [4]:
# config = {"configurable": {"session_id": "abc2"}}

In [6]:
# with_message_history = RunnableWithMessageHistory(chain, get_session_history)
# config = {"configurable": {"session_id": "abc5"}}
# response = with_message_history.invoke(
#     [HumanMessage(content="Hi! I'm Jim")],
#     config=config,
# )

# response

### Chat session Test

In [1]:
import uuid
from langserve import RemoteRunnable

chat = RemoteRunnable("http://localhost:8001/")
session_id = str(uuid.uuid4())


In [3]:
chat.invoke({"human_input": "hi, my name is Man, I like pizza. Ready to start?"}, {'configurable': { 'session_id': session_id } })

"\nGreat to hear that you're excited about data science, Man! 🍕 Before we dive into the topic, can you tell me a bit more about why you're interested in it? What do you hope to achieve or learn from this journey? 🤔"

In [10]:
chat.invoke({"human_input": "How can I get the average cat size from my data? I have all sorts of animals here"}, {'configurable': { 'session_id': session_id } })

'AI: 😊 Hey Man, cool to hear that you have a variety of pets! 🐱 Now, let\'s focus on your question. To calculate the average cat size in your data, you can use a technique called "mean calculation". Can you tell me more about your data and what you have so far?'

In [12]:
chat.invoke({"human_input": "what if I need the heaviest cat?"}, {'configurable': { 'session_id': session_id } })

"AI: Ah, great question! To find the heaviest cat in your dataset, you can use the `sort_values()` function to sort your data by the 'cat_weight' column in descending order (i.e. from highest to lowest). Then, you can take the first value in the sorted list, which will be the heaviest cat. Here's an example:\n```python\nimport pandas as pd\n\n# assume your dataset is stored in a variable called 'df'\nheaviest_cat = df['cat_weight'].sort_values(by='cat_weight', ascending=False).iloc[0]\nprint(heaviest_cat)\n```\nThis will give you the heaviest cat in your dataset. If you want to find the heaviest cat for a specific group of cats (e.g. by breed), you can use the `groupby()` function to group them by some characteristic and then calculate the heaviest weight within each group.\n\nBut wait, what about the other animals in your dataset? Do you have any interesting insights or patterns you'd like to explore? 😺"

In [14]:
chat.invoke({"human_input": "do you remember my favorite food?"}, {'configurable': { 'session_id': session_id } })

"AI: Of course! 🙅\u200d♂️ Your favorite food is pizza. Is there anything else you'd like to learn or work on today related to data science?"

### Message trimming

In [3]:
import json
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, trim_messages
from langchain_community.llms import Ollama

model = Ollama(model="llama2")

In [15]:
with open(f'chat_histories/{session_id}.json') as f:
    content = json.load(f)
messages = []
for message in content:
    if message['type'] == 'ai':
        messages.append(AIMessage(content=message['data']['content']))
    else:
        messages.append(HumanMessage(content=message['data']['content']))


trimmer = trim_messages(
    max_tokens=300,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=True,
    # start_on="human",
)

trimmer.invoke(messages)

[HumanMessage(content='what if I need the heaviest cat?'),
 AIMessage(content="AI: Ah, great question! To find the heaviest cat in your dataset, you can use the `sort_values()` function to sort your data by the 'cat_weight' column in descending order (i.e. from highest to lowest). Then, you can take the first value in the sorted list, which will be the heaviest cat. Here's an example:\n```python\nimport pandas as pd\n\n# assume your dataset is stored in a variable called 'df'\nheaviest_cat = df['cat_weight'].sort_values(by='cat_weight', ascending=False).iloc[0]\nprint(heaviest_cat)\n```\nThis will give you the heaviest cat in your dataset. If you want to find the heaviest cat for a specific group of cats (e.g. by breed), you can use the `groupby()` function to group them by some characteristic and then calculate the heaviest weight within each group.\n\nBut wait, what about the other animals in your dataset? Do you have any interesting insights or patterns you'd like to explore? 😺"),
 